In [2]:
import torch
import numpy as np

BATCH_SIZE = 1

# Input image dimensions
input_height, input_width, input_channels = 227, 227, 3

# Sample input data (you should replace this with your actual input data)
sample_input = np.random.randn(BATCH_SIZE, input_height, input_width, input_channels)

# Weight Generation
mean = 0
std = 0.01

features_weight = []
features_weight.append(torch.normal(mean=mean, std=std, size=(96, 3, 11, 11)))
features_bias = []
features_bias.append(torch.normal(mean=mean, std=std, size=(96,)))

In [4]:
from torch import nn
import torch.nn.init as init
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0)
        
        self.conv1.weight = nn.Parameter(features_weight[0])
        self.conv1.bias = nn.Parameter(features_bias[0])
        
    def forward(self, x):
        x = self.conv1(x)
        
        return x

import torch
import numpy as np

net = AlexNet()

input_data = sample_input.reshape(-1)
input_data = input_data.reshape(BATCH_SIZE, 3, 227, 227)
input_data = torch.from_numpy(input_data).float()

result2 = net.forward(input_data)
pred = torch.argmax(result2, dim=1)
print(pred)

tensor([[[23, 41,  3,  ...,  5, 84, 88],
         [83, 12,  4,  ..., 82, 83, 29],
         [36, 60, 69,  ..., 88, 50, 89],
         ...,
         [90, 55, 82,  ..., 40, 63, 11],
         [59, 64, 75,  ..., 30, 94, 38],
         [86, 83, 33,  ..., 47, 83, 38]]])


In [5]:
import numpy as np

# Define the convolutional layer function
class ConvLayer():
    def __init__(self, input_shape, num_filters, kernel_size, stride, padding, weight, bias):
        super(ConvLayer, self).__init__()
        self.batch_size, self.input_height, self.input_width, self.input_channels = input_shape
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.kernel = weight.transpose(3, 2, 1, 0)
        self.bias = bias.reshape((1,1,1,num_filters))
    
    def conv_forward(self, input_volume):
        
        output_height = (self.input_height - self.kernel_size + 2 * self.padding) // self.stride + 1
        output_width = (self.input_width - self.kernel_size + 2 * self.padding) // self.stride + 1
        
        input_volume = np.pad(input_volume, ((0, 0), (self.padding, self.padding), (self.padding, self.padding), (0, 0)), mode='constant')

        output_volume = np.zeros((self.batch_size, output_height, output_width, self.num_filters))
        
        for i in range(output_height):
            for j in range(output_width):
                for f in range(self.num_filters):
                    h_start = i * self.stride
                    h_end = h_start + self.kernel_size
                    w_start = j * self.stride
                    w_end = w_start + self.kernel_size
                    receptive_field = input_volume[:, h_start:h_end, w_start:w_end, :]
                    for a in range(self.batch_size):
                        temp = 0
                        for b in range(self.kernel_size):
                            for c in range(self.kernel_size):
                                for d in range(self.input_channels):
                                    temp += receptive_field[a,b,c,d] * self.kernel[b,c,d,f]
                        output_volume[a, i, j, f] = temp + self.bias[:,:,:,f]
                    # output_volume[:, i, j, f] = np.sum(receptive_field * self.kernel[:, :, :, f], axis=(1, 2, 3)) + self.bias[:,:,:,f]
        
        return output_volume
    
class AlexNet_low():
    def __init__(self, batch_size):
        self.batch_size = batch_size
        
        self.conv1 = ConvLayer((batch_size, 227, 227, 3), 96, 11, 4, 0, features_weight[0].numpy(), features_bias[0].numpy())
        
        
    def forward(self, x):
        x = self.conv1.conv_forward(x)
        
        return x
    
net_low = AlexNet_low(batch_size=BATCH_SIZE)

result = net_low.forward(sample_input)
pred = np.argmax(result, axis=1)
print(pred)

/tmp/ipykernel_9932/2378005889.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  output_volume[a, i, j, f] = temp + self.bias[:,:,:,f]


[[[22 39 41 ... 34 47 28]
  [51 17 36 ... 45 30 44]
  [36 32 21 ... 38  7  8]
  ...
  [24 16 42 ...  5 19  9]
  [11 11 13 ... 43 10 45]
  [ 5  9 29 ... 44 40 40]]]


In [8]:
import numpy as np

position = []
for i in range(227):
    if i < 113:
        position.append([1]*227)
    else:
        position.append([0]*227)

# for row in position:
#     print(row)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [11]:
import numpy as np

# Define the convolutional layer function
class ConvLayer():
    def __init__(self, input_shape, num_filters, kernel_size, stride, padding, weight, bias, position):
        super(ConvLayer, self).__init__()
        self.batch_size, self.input_height, self.input_width, self.input_channels = input_shape
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.kernel = weight.transpose(3, 2, 1, 0)
        self.bias = bias.reshape((1,1,1,num_filters))
        
        self.position = position
    
    def conv_forward(self, input_volume):
        
        output_height = (self.input_height - self.kernel_size + 2 * self.padding) // self.stride + 1
        output_width = (self.input_width - self.kernel_size + 2 * self.padding) // self.stride + 1
        
        input_volume = np.pad(input_volume, ((0, 0), (self.padding, self.padding), (self.padding, self.padding), (0, 0)), mode='constant')

        output_volume = np.zeros((self.batch_size, output_height, output_width, self.num_filters))
        
        for i in range(output_height):
            for j in range(output_width):
                for f in range(self.num_filters):
                    h_start = i * self.stride
                    h_end = h_start + self.kernel_size
                    w_start = j * self.stride
                    w_end = w_start + self.kernel_size
                    receptive_field = input_volume[:, h_start:h_end, w_start:w_end, :]
                    
                    if sum(sum(row[w_start:w_end]) for row in self.position[h_start:h_end]) != 0:
                        for a in range(self.batch_size):
                            temp = 0
                            for b in range(self.kernel_size):
                                for c in range(self.kernel_size):
                                    for d in range(self.input_channels):
                                        temp += receptive_field[a,b,c,d] * self.kernel[b,c,d,f]
                            output_volume[a, i, j, f] = temp + self.bias[:,:,:,f]
                    else:
                        output_volume[:, i, j, f] = np.sum(receptive_field * self.kernel[:, :, :, f], axis=(1, 2, 3)) + self.bias[:,:,:,f]
        
        return output_volume
    
class AlexNet_half():
    def __init__(self, batch_size):
        self.batch_size = batch_size
        
        self.conv1 = ConvLayer((batch_size, 227, 227, 3), 96, 11, 4, 0, features_weight[0].numpy(), features_bias[0].numpy(), position)
        
        
    def forward(self, x):
        x = self.conv1.conv_forward(x)
        
        return x
    
net_half = AlexNet_half(batch_size=BATCH_SIZE)

result_half = net_half.forward(sample_input)
pred = np.argmax(result_half, axis=1)
print(pred)

/tmp/ipykernel_9932/285894905.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  output_volume[a, i, j, f] = temp + self.bias[:,:,:,f]


[[[22 39 41 ... 34 47 28]
  [51 17 36 ... 45 30 44]
  [36 32 21 ... 38  7  8]
  ...
  [24 16 42 ...  5 19  9]
  [11 11 13 ... 43 10 45]
  [ 5  9 29 ... 44 40 40]]]
